In [3]:
import urllib
import os
import pandas as pd

In [4]:
province = ["http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/arezzo.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/firenze.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/grosseto.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/livorno.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/lucca.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/massa.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/pisa.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/pistoia.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/prato.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/siena.pdf"]

In [12]:
def scarica_pdf(link):
    urllib.urlretrieve(link, os.path.join(os.getcwd(), "PDF", os.path.basename(link)))
    

In [13]:
for provincia in province:
    scarica_pdf(provincia)

<h2> proviamo a effettuare la trasformazione del pdf in txt attraverso PDFminer </h2>

In [5]:
from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = file(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

In [70]:
#convert(os.path.basename(province[0])).split("\n\n")

<h2> esito negativo, utilizziamo TABULA </h2>

In [71]:
#os.listdir(os.getcwd())

<h2> aggreghiamo i dati estratti con tabula </h2>

In [15]:
toscana = pd.DataFrame(columns = [u'provincia', u'comune', u'Ragione sociale', u'indirizzo', u'civico', u'destinazione', u'sup_coperta'])

In [17]:
for item in ['tabula-arezzo.csv',
 'tabula-firenze.csv',
 'tabula-grosseto.csv',
 'tabula-livorno.csv',
 'tabula-lucca.csv',
 'tabula-massa.csv',
 'tabula-pisa.csv',
 'tabula-pistoia.csv',
 'tabula-prato.csv',
 'tabula-siena.csv']:
    url = os.path.join(os.getcwd(), "Tabula", item)
    toscana = pd.concat([toscana, pd.read_csv(url, encoding='utf-8', sep=",")])

<h2> rimuoviamo le intestazioni di colonna in eccesso </h2>

In [18]:
toscana = toscana[toscana["provincia"] != "provincia"]

In [19]:
#si risolve una stranezza unicode di importazione tabula
for columns in range(0,toscana.shape[1]):
    for rows in range(0,toscana.shape[0]):
        try: 
            unicode(toscana.iloc[rows, columns])
        except:
            for caracter in toscana.iloc[rows, columns]:
                try:
                    unicode(caracter)
                except:
                    toscana.iloc[rows, columns] = toscana.iloc[rows, columns].replace(caracter, "X")

<h2> rimuoviamo le interruzioni di linea all'interno delle celle create da Tabula </h2>

In [20]:
def replace(x):
    if type(x) == str or type(x) == unicode:
        x = x.replace('\r', '')
    else:
        x = x[0].replace('\r', '')
    return x
    
toscana["indirizzo"] = toscana["indirizzo"].map(lambda x: x.replace('"', ''))
toscana["indirizzo"] = toscana["indirizzo"].map(lambda x: replace(x))

toscana["Ragione sociale"] = toscana["Ragione sociale"].map(lambda x: x.replace('"', ''))
toscana["Ragione sociale"] = toscana["Ragione sociale"].map(lambda x: replace(x))


<h2> prima prova di geolocalizzazione </h2>

In [25]:
import geopy

In [75]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Firenze Via di Novoli 23")
print(location.address)

42/23, Via di Novoli, Careggi, Firenze, FI, TOS, 50127, Italia


In [76]:
location.latitude, location.longitude

(43.793649, 11.2202068)

In [44]:
def coordinate(via):
    try:
        location = geolocator.geocode(via)
        return location.latitude, location.longitude
    except:
        pass
    

In [45]:
toscana["geo"] = toscana["comune"] + " " + toscana["indirizzo"]

In [46]:
toscana["coordinate"] = toscana["geo"].map(lambda x: coordinate(x))

In [65]:
def estrai_lat_long(x, posizione):
    try:
        return x[posizione]
    except:
        pass


toscana["latitudine"] = toscana["coordinate"].map(lambda x: estrai_lat_long(x,0))

In [66]:
toscana["longitudine"] = toscana["coordinate"].map(lambda x: estrai_lat_long(x,1))

In [74]:
toscana.to_excel("toscana.xlsx")

In [69]:
toscana.to_csv("toscana.csv", encoding = "utf-8")

In [89]:
nulle = toscana["coordinate"].isnull().sum() 
casi = len(toscana["coordinate"])

In [94]:
float(nulle) / casi

0